# relax_sim

In [ ]:
DESCRIPTION = """
Panos Manganaris, Kat Nykiel
Simtool for the automated generation of Raman and IR Spectra from QE vc-relax>scf>ph>dynmat pipeline
"""

## Define Inputs and Outputs
Define default values which can be overwritten at runtime -- input parameters should NOT be interdependent.

loosely interdependent inputs can be provided using frontend widgets

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

# POSCAR string taken from frontend
struct_dict:
    type: Dict
    description: User selected structure as dictionary

# pp_class:
#     type: Choice
#     description: Class of selected pseudopotentials. SSSP - standard solid-state library optimized for precision and efficiency. ONCV - optimized norm-conserving vanderbilt. USPP - ultra-soft pseudopotentials. PAW - projector augmented wave.
#     options: ['SSSP','ONCV','USPP','PAW']
#     value: 'ONCV'
        
# xc_functional:
#     type: Choice
#     description: Exchange-correlation functional - eg PBE, BLYP etc. Most of the pseudopotentials provided recommend using PBE. See Modules/funct.f90 for detailed descriptions
#     options: ['pz','pw91','blyp','pbe']
#     value: 'pbe'
        
ecutwfc:
    type: Number
    description: Kinetic energy cutoff for wavefunctions
    value: 80
    min: 40
    max: 200
    units: Ry

kpoints:
    type: Number
    description: Number of kpoints along each reciprocal lattice vector
    value: 6
    min: 1
    max: 20

# smearing:
#     type: Choice
#     description: Setting the extent to which there is metal-like sharing of electrons
#     options: ['smearing', 'fixed']
#     value: 'fixed'


In [ ]:
%%yaml OUTPUTS

spectra:
    type: Dict
    description: Dictionary of pandas Series of dynamat.x output containing mode numbers and frequencies of IR and Raman Spectra

logreport:
    type: Text
    description: Contents of the run logfile populated according to loglevel
        
relaxed_struct:
    type: Dict
    description: Dictionary of pymatgen structure object, post-relaxation
    
total_energy:
    type: Number
    description: Total energy of the system, calculated in the SCF step
        
stress_tensor:
    type: Array
    description: Tensor of the stresses, calculated in the SCF step
    units: kbar
        
atomic_forces: 
    type: Array
    description: Forces acting on atoms (cartesian axes)
    units: Ry/au
        
chemical_formula:
    type: Text
    description: Chemical formula of the pyamtgen structure object
        
# raman_cutoff:
#     type: Number
#     description: Highest frequency with a non-zero Raman coefficient

# IR_cutoff:
#     type: Number
#     description: Highest frequency with a non-zero IR coefficient

# dielectric constant:
#     type: Number
#     description: Dielectric constant, computed in the PHonon code of QE, in units of [[TODO]]

## TODO: we want some other metric of convergence of the spectra (number of negative frequencies?)

In [ ]:
EXTRA_FILES = ["./pseudo", "./backend.py"]

## define simtool dependencies

In [ ]:
import logging
logfmt = '[%(levelname)s] %(asctime)s - %(message)s'
logging.basicConfig(filename='run.log', level=logging.debug, datefmt="%Y-%m-%d %H:%M:%S", format=logfmt)

In [ ]:
#cli utilities
import io
import shutil
import subprocess

#nanohub utilities
import hublib.use
import fileinput
from simtool import DB, parse

#automate retreival of crystal structures and pseudopotentials from Materials Project
from pymatgen.core import Structure, Lattice, Element, Composition
import openbabel # automates translation of pymatgen structs to useful file formats
#process and categorize crystal structures

#misc tools
import os
import re
import math
import pandas as pd
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.16f}'.format}, suppress=False)

#%use espresso-6.2.1
#Apparently not functioning? sufficient to declare espresso-6.8 in submit call
from backend import *

In [ ]:
from simtool import getValidatedInputs

defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)
#papermill adds injected parameters cell after this cell containing run settings
#Don't intake passwords/keys/private info they cannot be deleted from RUNS

## Create Global Database Sessions

In [ ]:
db = DB(OUTPUTS)
pp_file_names = os.listdir(f"./pseudo/pseudo_{pp_class}/")
ppdb = {re.split('\.|\_',pp)[0]:pp for pp in pp_file_names}
logging.info(f'used pseudo-potentials of type {pp_class} to construct ppdb')

## Derive Control Parameters from Inputs


In [ ]:
# Define simulation settings compatible with Raman calculation
pp_class = 'ONCV'

# Define logging and queue settings
numCPUs = 1 # Number of processors for MPI execution
walltime = "01:00:00" # Maximum time to wait for reduced job load on cluster in HH:MM:SS time format
storage = 'none' # Set the ammount of disk activity. higher settings require more ram. Only change this if you need wavefunctions or the xml output from scf executions. the high setting is not provided. debug your wavefunctions elsewhere.

# Define simulation settings
epsil = True # Calculate dielectric constant for non-metal where q=0
lraman = True # Calculate non-resonant Raman coefficients
smearing = 'fixed' # Setting the extent to which there is metal-like sharing of electrons

# Define ecutrho based on ecutwfc and the class of pseudopotential selected
# ecutrho: kinetic energy cutoff for charge density and potential
if pp_class == 'USPP':
    ecutrho = 8*ecutwfc
else:
    ecutrho = 4*ecutwfc

## Validating/Correcting User Defined Inputs

In [ ]:
struct = Structure.from_dict(struct_dict)
logging.debug(f"the struct is made! {struct.sites}")
verify_site_fidelity(struct)

## Prepare input files to simulation pipline in quantum espresso .in format

#### insert validated inputs into the predetermined format strings:
- TODO:  set nosym=True as a setting?

In [ ]:
bp = BlockPrinter(struct, pp_class)

In [ ]:
ph_input = f"""
Normal modes for {vcomp.reduced_formula}
 &inputph
  tr2_ph=1.0d-14,
  prefix='{vcomp.reduced_formula}',
  {amass_block}outdir='./'
  epsil=.{epsil}.,
  lraman=.{lraman}.,
  trans=.true.,
  asr=.true.,
  fildyn='dmat.{vcomp.reduced_formula}'
  ! ldisp=.true.
 /
 0.0 0.0 0.0
"""

dm_input = f"""
&input fildyn='dmat.{vcomp.reduced_formula}', asr='zero-dim' /
"""
logging.debug(f"{ph_input}")

## Write Input Files. Name Output Files. Prepare to Assign Validated Outputs

In [ ]:
# fixed starting point input file generation
ph_input_file = open(f"{vcomp.reduced_formula}.ph.in", "w")
dm_input_file = open(f"{vcomp.reduced_formula}.dm.in", "w")

ph_input_file.write(ph_input)
dm_input_file.write(dm_input)

ph_input_file.close()
dm_input_file.close()

In [ ]:
# output file generation
vcr_output_file = open(f"{vcomp.reduced_formula}.vc-relax.out", "w")
scf_output_file = open(f"{vcomp.reduced_formula}.scf.out", "w")
ph_output_file = open(f"{vcomp.reduced_formula}.ph.out", "w")
#dmat_file = open(f"dmat.{vcomp.reduced_formula}", "w")
dm_output_file = open(f"{vcomp.reduced_formula}.dm.out", "w")

## Prepare Psudopotential args for Run

In [ ]:
pp_args = ""
for pp in pps:
    pp_args += f"-i pseudo/pseudo_{pp_class}/{pp} "

## Assemble Job Commands and Sequentially Submit to Cluster
#### First perform variable cell relaxation and receive optimized structure

In [ ]:
COMMANDvcr = f"espresso-6.8_pw > {vcr_output_file.name}"
SUBMITvcr = f"submit -n {numCPUs} -w {walltime} -e QE_DISABLE_GGA_PBE=0 --runName {vcomp.reduced_formula}vcr {COMMANDvcr} {pp_args} -i {vcr_input_file.name} "
logging.info("reached cell relaxation...")
spvcr = subprocess.run(SUBMITvcr.split(), capture_output=True, text=True)
spvcr_out = """""".join(spvcr.stdout)
spvcr_err = """""".join(spvcr.stderr)
logging.debug(" ".join(spvcr.args))
logging.info(f"""\nprocess output:\n{spvcr_out}\n""")
logging.debug(f"""\nprocess err out:\n{spvcr_err}\n""")
vcr_output_file.close()
#db.save('vcrstdout', spvcr) #cannot save artbitrary objects as outputs.

In [ ]:
matrix = [[16.714103908  ,    -0.000106536      , 0.000000012 ],
                [-0.000106465      ,16.713831819  ,     0.000000009 ],
                [ 0.000000012    ,   0.000000009    ,  16.713839428 ]]
np.linalg.eig(matrix)

# Extract the new structure from the vc-relax calculation

In [ ]:
# extract structure data from vc-relax.out and save to scf input
matrix_start = "CELL_PARAMETERS"
atpos_start = "ATOMIC_POSITIONS"
atpos_end = "End final coordinates"
matrix_inds = []
atpos_inds = []
with open(f"{vcr_output_file.name}", "rt") as vcrout:
    vcr_lines = vcrout.readlines()
    
#raise exception for bad file
if not vcr_lines:
    raise IndexError("vcrelax calculation did not output anything to file. It probably failed to run")
else:
    preamble = "".join(vcr_lines[14:40])
    logging.debug(f"""The first meaningful lines of vc relaxation output are: {preamble}\n""")

try:
    # Getting structure block line locations in the file
    for ind, line in enumerate(vcr_lines):
        if matrix_start in line:
            logging.debug(f"adding line {ind} to cell matrix reference locations")
            matrix_inds.append(ind)  
        if atpos_start in line:
            logging.debug(f"adding line {ind} to site reference locations")
            atpos_inds.append(ind)
        if atpos_end in line:
            logging.debug(f"line {ind} is the end of structure info")
            end = ind
    
    # get the new cell parameters
    relaxed_cell_parameters_block = """"""
    for line in [lines for lines in vcr_lines[matrix_inds[-1]+1:atpos_inds[-1]-1]]:
        logging.debug("looping though final cell matrix lines")
        relaxed_cell_parameters_block += line
    
    # get the new atomic positions
    if end:
        relaxed_atomic_positions_block = """"""
        for line in [lines for lines in vcr_lines[atpos_inds[-1]+1:end]]:
            logging.debug("looping though final atomic sites lines")
            relaxed_atomic_positions_block += line
    else:
        logging.error("Variable Cell Relaxation Failed to Converge! Try increasing ecutwfc to at least 100. The calculation is proceeding with most relaxed structure found.")
        end_lines_list = [line for line in vcr_lines[atpos_inds[-1]+1:-1]]
        relaxed_atomic_positions_block = """"""
        for line in end_lines_list:
            if line != "\n" :
                logging.debug("looping though final atomic sites lines")
                relaxed_atomic_positions_block += line                 
                 
    logging.info(f"""The Lattice Parameters after relaxation are:
    {relaxed_cell_parameters_block}""")
    logging.info(f"""The atomic sites after relaxation are:
    {relaxed_atomic_positions_block}""")
except:
    raise ValueError(f"{vcr_output_file.name} contains lines, but no reference to a relaxed structure. pw.x failed for some reason")

scf_input = f"""
&CONTROL
  calculation  = "scf",
  prefix       = "{vcomp.reduced_formula}",
  pseudo_dir   = "./",
  outdir       = "./",
  tstress      = .TRUE.,
  tprnfor      = .TRUE.,
/
&SYSTEM
  ibrav=0, celldm(1) =1, nat = {nat_num}, ntyp= {ntyp_num},
  occupations={smearing}, {"smearing='marzari-vanderbilt', degauss=0.02," if smearing == 'smearing' else ""}
  ecutwfc ={ecutwfc}, ecutrho = {ecutrho},
  input_dft = {xc_functional},
/
&ELECTRONS
  mixing_mode='plain'
  mixing_beta = 0.5,
  startingwfc='random',
  conv_thr =  1.0d-8
/
CELL_PARAMETERS (alat= 1.00000000)
{relaxed_cell_parameters_block}
ATOMIC_SPECIES
{atomic_species_block}
ATOMIC_POSITIONS (crystal)
{relaxed_atomic_positions_block}
K_POINTS (automatic)
{kpoints_block}
"""

# write to the file
scf_input_file = open(f"{vcomp.reduced_formula}.scf.in", "w")
scf_input_file.write(scf_input)
scf_input_file.close()

#### Next perform self consistent field calculation with new structure to optimize wavefunction

In [ ]:
COMMANDscf = f"espresso-6.8_pw > {scf_output_file.name}"
SUBMITscf = f"submit -n {numCPUs} -w {walltime} -e QE_DISABLE_GGA_PBE=0 --runName {vcomp.reduced_formula}scf {COMMANDscf} {pp_args} -i {scf_input_file.name}"
logging.info("reached self consistent field calculation...")
spscf = subprocess.run(SUBMITscf.split(), capture_output=True, text=True)
spscf_out = """""".join(spscf.stdout)
spscf_err = """""".join(spscf.stderr)
logging.debug(" ".join(spscf.args))
logging.info(f"""\nprocess output:\n{spscf_out}\n""")
logging.debug(f"""\nprocess err out:\n{spscf_err}\n""")
scf_output_file.close()
#db.save('scfstdout', spscf)

#### Next compute vibrational frequencies
ph.x takes as inputs:
1. compound.ph.in file
2. compound.scf.out file

produces outputs:
1. compound.ph.out file
2. dmat.compound file

In [ ]:
COMMANDph = f"espresso-6.8_ph > {ph_output_file.name}"
extra_inargs = f"-i {vcomp.reduced_formula}.xml -i {vcomp.reduced_formula}.save"
SUBMITph = f"submit -n {numCPUs} -w {walltime} -e QE_DISABLE_GGA_PBE=0 --runName {vcomp.reduced_formula}ph {extra_inargs} {COMMANDph} -in {ph_input_file.name} {pp_args}"
logging.info("reached phonon calculation...")
spph = subprocess.run(SUBMITph.split(), capture_output=True, text=True)
spph_out = """""".join(spph.stdout)
spph_err = """""".join(spph.stderr)
logging.debug(" ".join(spph.args))
logging.info(f"""\nprocess output:\n{spph_out}\n""")
logging.debug(f"""\nprocess err out:\n{spph_err}\n""")
ph_output_file.close()
#db.save('phstdout', spph)

#### Next extract phonon spectra
dynmat.x takes as inputs:
1. compound.dm.in
2. dmat.compound

dmat.compound may be malformed... Why this happens, even amongst compositoins in the same spacegroup simply with variably defined unit cells is unknown to me.

produces output:
1. compound.dm.out

This output contains the spectrum tensor and can be used to plot the spectrum

In [ ]:
try:
    with open(f"dmat.{vcomp.reduced_formula}", 'r') as dynmat:
        dmtext = dynmat.read()
    logging.info(dmtext)
except:
    logging.critical(f"dmat.{vcomp.reduced_formula} is empty. ph.x likely crashed")

In [ ]:
COMMANDdm = f"espresso-6.8_dynmat > {dm_output_file.name}"
extra_inargs = f"-i dmat.{vcomp.reduced_formula}"
SUBMITdm = f"submit -n {numCPUs} -w {walltime} -e QE_DISABLE_GGA_PBE=0 --runName {vcomp.reduced_formula}dm {extra_inargs} {COMMANDdm} -in {dm_input_file.name}" 
logging.info("reached dynamical matrix calculation...")
spdm = subprocess.run(SUBMITdm.split(), capture_output=True, text=True)
spdm_out = """""".join(spdm.stdout)
spdm_err = """""".join(spdm.stderr)
logging.debug(" ".join(spdm.args))
logging.info(f"""\nprocess output:\n{spdm_out}\n""")
logging.debug(f"""\nprocess err out:\n{spdm_err}\n""")
#dmat_file.close() #might be overwritting the qe's attempt to output this file automatically?
dm_output_file.close()
#db.save('dmstdout', spdm)

## Parse Output Files for Declared Results

In [ ]:
with open(f"{dm_output_file.name}", "rt") as resultfile:
    results = resultfile.readlines()

try:
    results_start = "mode"
    results_end = "DYNMAT"
    for ind, line in enumerate(results):
        if results_start in line:
            start = ind
        if results_end in line:
            end = ind
    
    spectra_data = "".join(results[start:end]).replace("#"," ")

    logging.info(f"""The Predicted Spectrographs for {sa.get_crystal_system()} {vcomp.reduced_formula} are: {spectra_data}""")
except:
    logging.error(f"{dm_output_file.name} may not contain a modes and frequencies card. dynmat.x likely failed to produce it. dmat.{vcomp.reduced_formula} may be malformed")
    pass


In [ ]:
with open(f"ZnS-Copy1.dm.out", "rt") as resultfile:
    results = resultfile.readlines()

try:
    results_start = "mode"
    results_end = "DYNMAT"
    for ind, line in enumerate(results):
        if results_start in line:
            start = ind
        if results_end in line:
            end = ind
    
    spectra_data = "".join(results[start:end]).replace("#"," ")

    logging.info(f"""The Predicted Spectrographs for {sa.get_crystal_system()} {vcomp.reduced_formula} are: {spectra_data}""")
except:
    logging.error(f"{dm_output_file.name} may not contain a modes and frequencies card. dynmat.x likely failed to produce it. dmat.{vcomp.reduced_formula} may be malformed")
    pass


In [ ]:
# Build pymatgen relaxed structure object
lattice_matrix = [float(s)*0.52917612569 for s in relaxed_cell_parameters_block.split()]
lattice = Lattice(matrix=lattice_matrix)
atom_list = relaxed_atomic_positions_block.split()[::4]
position_list = relaxed_atomic_positions_block.split()
del position_list[0::4]
site_list = [float(a) for a in position_list]
site_matrix = [site_list[i:i + 3] for i in range(0, len(site_list), 3)]
relaxed_struct_object = Structure(lattice, atom_list, site_matrix)

# Extract sim2L outputs
relaxed_struct = relaxed_struct_object.as_dict()
formula = relaxed_struct_object.formula


In [ ]:
# Extract SCF outputs
scf_output_file = open(f"{vcomp.reduced_formula}.scf.out", "r")
lines = scf_output_file.readlines()
scf_output_file.close()
for i, line in enumerate(lines):
    if '!' in line:
        total_energy = float(line.split()[4])
    elif 'kbar' in line:
        stress_tensor = np.array([lines[i+1].split()[3:6],lines[i+2].split()[3:6],lines[i+3].split()[3:6]]).astype(float)
    elif 'Forces' in line:
        force_matrix = []
        for atom in range(nat_num):
            force_matrix.append(lines[i+2+atom].split()[6:9])
        atomic_forces = np.array(force_matrix).astype(float)

In [ ]:
# Extract Raman and IR Cutoffs, and dielectric constant


### Remove unwanted directories

In [ ]:
# clear directories which take up a lot of storage
# clear directories which take up a lot of storage
# os.system('rm -r pseudo/') -- This doesn't work, kat. It just deletes the pseudo repository 
# os.system('rm -r *.save')

### Assign results to output variables/hardcopy plot files

In [ ]:
try:
    result_stream = io.StringIO(spectra_data)
    result_stream.seek(0)
    spectradf = pd.read_csv(result_stream, error_bad_lines=False, sep="\s+|\t+|\s+\t+|\t+\s")
except:
    pass

In [ ]:
with open("run.log", 'r') as logfile:
    logtext = logfile.read()
db.save('logreport', logtext)

In [ ]:
try: 
    db.save('spectra', spectradf.to_dict())
except:
    raise ValueError(f"No spectra dataframe was produced from processing {dm_output_file.name}")